In [1]:
import org.apache.spark.ml.{PipelineModel, Pipeline}
import org.apache.spark.ml.classification.{DecisionTreeClassifier,
                                          RandomForestClassifier, RandomForestClassificationModel}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{VectorAssembler, VectorIndexer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import scala.util.Random

Intitializing Scala interpreter ...

Spark Web UI available at http://localhost:4040
SparkContext available as 'sc' (version = 2.4.6, master = local[*], app id = local-1601226835588)
SparkSession available as 'spark'


import org.apache.spark.ml.{PipelineModel, Pipeline}
import org.apache.spark.ml.classification.{DecisionTreeClassifier, RandomForestClassifier, RandomForestClassificationModel}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{VectorAssembler, VectorIndexer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import scala.util.Random


In [2]:
val dataWithoutHeader = spark.read.
    option("inferSchema", true).
    option("header", false).
    csv("Data/covtype.data")

dataWithoutHeader: org.apache.spark.sql.DataFrame = [_c0: int, _c1: int ... 53 more fields]


In [3]:
dataWithoutHeader.first

res0: org.apache.spark.sql.Row = [2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5]


In [4]:
dataWithoutHeader.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)
 |-- _c8: integer (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)
 |-- _c15: integer (nullable = true)
 |-- _c16: integer (nullable = true)
 |-- _c17: integer (nullable = true)
 |-- _c18: integer (nullable = true)
 |-- _c19: integer (nullable = true)
 |-- _c20: integer (nullable = true)
 |-- _c21: integer (nullable = true)
 |-- _c22: integer (nullable = true)
 |-- _c23: integer (nullable = true)
 |-- _c24: integer (nullable = true)
 |-- _c25: integer (nullable = true)
 |-- _c26: integer (nullable = true)
 |-- _

In [5]:
val colNames = Seq(
    "Elevation","Aspect","Slope",
    "Horizontal_Distance_To_Hydriology","Vertical_Distance_To_Hydrology",
    "Horizontal_Distance_To_Roadways",
    "Hillshade_9am","Hillshade_Noon","Hillshade_3pm",
    "Horizontal_Distance_To_Fire_Points") ++
    ((0 until 4).map(i => s"Wilderness_Area_$i")) ++
    ((0 until 40).map(i=> s"Soil_Type_$i")) ++ Seq("Cover_Type")

val data = dataWithoutHeader.toDF(colNames:_*).withColumn("Cover_Type",$"Cover_Type".cast("double"))

data.printSchema()

root
 |-- Elevation: integer (nullable = true)
 |-- Aspect: integer (nullable = true)
 |-- Slope: integer (nullable = true)
 |-- Horizontal_Distance_To_Hydriology: integer (nullable = true)
 |-- Vertical_Distance_To_Hydrology: integer (nullable = true)
 |-- Horizontal_Distance_To_Roadways: integer (nullable = true)
 |-- Hillshade_9am: integer (nullable = true)
 |-- Hillshade_Noon: integer (nullable = true)
 |-- Hillshade_3pm: integer (nullable = true)
 |-- Horizontal_Distance_To_Fire_Points: integer (nullable = true)
 |-- Wilderness_Area_0: integer (nullable = true)
 |-- Wilderness_Area_1: integer (nullable = true)
 |-- Wilderness_Area_2: integer (nullable = true)
 |-- Wilderness_Area_3: integer (nullable = true)
 |-- Soil_Type_0: integer (nullable = true)
 |-- Soil_Type_1: integer (nullable = true)
 |-- Soil_Type_2: integer (nullable = true)
 |-- Soil_Type_3: integer (nullable = true)
 |-- Soil_Type_4: integer (nullable = true)
 |-- Soil_Type_5: integer (nullable = true)
 |-- Soil_Typ

colNames: Seq[String] = List(Elevation, Aspect, Slope, Horizontal_Distance_To_Hydriology, Vertical_Distance_To_Hydrology, Horizontal_Distance_To_Roadways, Hillshade_9am, Hillshade_Noon, Hillshade_3pm, Horizontal_Distance_To_Fire_Points, Wilderness_Area_0, Wilderness_Area_1, Wilderness_Area_2, Wilderness_Area_3, Soil_Type_0, Soil_Type_1, Soil_Type_2, Soil_Type_3, Soil_Type_4, Soil_Type_5, Soil_Type_6, Soil_Type_7, Soil_Type_8, Soil_Type_9, Soil_Type_10, Soil_Type_11, Soil_Type_12, Soil_Type_13, Soil_Type_14, Soil_Type_15, Soil_Type_16, Soil_Type_17, Soil_Type_18, Soil_Type_19, Soil_Type_20, Soil_Type_21, Soil_Type_22, Soil_Type_23, Soil_Type_24, Soil_Type_25, Soil_Type_26, Soil_Type_27, Soil_Type_28, Soil_Type_29, Soil_Type_30, Soil_Type_31, Soil_Type_32, Soil_Type_33, Soil_Type_34, Soil...

In [6]:
val Array(trainData, testData) = data.randomSplit(Array(0.9,0.1))
trainData.cache()
testData.cache()

trainData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Elevation: int, Aspect: int ... 53 more fields]
testData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Elevation: int, Aspect: int ... 53 more fields]
res3: testData.type = [Elevation: int, Aspect: int ... 53 more fields]


In [7]:
val inputCols = trainData.columns.filter(_ != "Cover_Type")
val assembler = new VectorAssembler().
    setInputCols(inputCols).
    setOutputCol("featureVector")


inputCols: Array[String] = Array(Elevation, Aspect, Slope, Horizontal_Distance_To_Hydriology, Vertical_Distance_To_Hydrology, Horizontal_Distance_To_Roadways, Hillshade_9am, Hillshade_Noon, Hillshade_3pm, Horizontal_Distance_To_Fire_Points, Wilderness_Area_0, Wilderness_Area_1, Wilderness_Area_2, Wilderness_Area_3, Soil_Type_0, Soil_Type_1, Soil_Type_2, Soil_Type_3, Soil_Type_4, Soil_Type_5, Soil_Type_6, Soil_Type_7, Soil_Type_8, Soil_Type_9, Soil_Type_10, Soil_Type_11, Soil_Type_12, Soil_Type_13, Soil_Type_14, Soil_Type_15, Soil_Type_16, Soil_Type_17, Soil_Type_18, Soil_Type_19, Soil_Type_20, Soil_Type_21, Soil_Type_22, Soil_Type_23, Soil_Type_24, Soil_Type_25, Soil_Type_26, Soil_Type_27, Soil_Type_28, Soil_Type_29, Soil_Type_30, Soil_Type_31, Soil_Type_32, Soil_Type_33, Soil_Type_34, ...

In [8]:
val assembledTrainData = assembler.transform(trainData)
assembledTrainData.select("featureVector").show(truncate = false)

+----------------------------------------------------------------------------------------------------+
|featureVector                                                                                       |
+----------------------------------------------------------------------------------------------------+
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1859.0,18.0,12.0,67.0,11.0,90.0,211.0,215.0,139.0,792.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1860.0,18.0,13.0,95.0,15.0,90.0,210.0,213.0,138.0,780.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1861.0,35.0,14.0,60.0,11.0,85.0,218.0,209.0,124.0,832.0,1.0,1.0])  |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1863.0,37.0,17.0,120.0,18.0,90.0,217.0,202.0,115.0,769.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1866.0,23.0,14.0,85.0,16.0,108.0,212.0,210.0,133.0,819.0,1.0,1.0]) |
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1867.0,20.0,15.0,108.0,19.0,120.0,208.0,206.0,132.0,808.0,1.0,1.0])|
|(54,[0,1,2,3,4,5,6,7,8,9,13,15],[1868.0,27.0,16.0,67.0,17.0,95.0,212.0,2

assembledTrainData: org.apache.spark.sql.DataFrame = [Elevation: int, Aspect: int ... 54 more fields]


In [9]:
val classifier = new DecisionTreeClassifier().
    setSeed(Random.nextLong()).
    setLabelCol("Cover_Type").
    setFeaturesCol("featureVector").
    setPredictionCol("prediction")

classifier: org.apache.spark.ml.classification.DecisionTreeClassifier = dtc_0e16dd4bea91


In [10]:
val model = classifier.fit(assembledTrainData)
println(model.toDebugString)

DecisionTreeClassificationModel (uid=dtc_0e16dd4bea91) of depth 5 with 49 nodes
  If (feature 0 <= 3053.5)
   If (feature 0 <= 2479.5)
    If (feature 3 <= 15.0)
     If (feature 12 <= 0.5)
      If (feature 23 <= 0.5)
       Predict: 4.0
      Else (feature 23 > 0.5)
       Predict: 3.0
     Else (feature 12 > 0.5)
      Predict: 6.0
    Else (feature 3 > 15.0)
     If (feature 16 <= 0.5)
      Predict: 3.0
     Else (feature 16 > 0.5)
      If (feature 9 <= 1345.0)
       Predict: 3.0
      Else (feature 9 > 1345.0)
       Predict: 4.0
   Else (feature 0 > 2479.5)
    If (feature 17 <= 0.5)
     If (feature 0 <= 2959.5)
      If (feature 15 <= 0.5)
       Predict: 2.0
      Else (feature 15 > 0.5)
       Predict: 3.0
     Else (feature 0 > 2959.5)
      Predict: 2.0
    Else (feature 17 > 0.5)
     If (feature 0 <= 2691.5)
      Predict: 3.0
     Else (feature 0 > 2691.5)
      If (feature 5 <= 1192.5)
       Predict: 5.0
      Else (feature 5 > 1192.5)
       Predict: 2.0
  Else (fe

model: org.apache.spark.ml.classification.DecisionTreeClassificationModel = DecisionTreeClassificationModel (uid=dtc_0e16dd4bea91) of depth 5 with 49 nodes


In [11]:
model.featureImportances.toArray.zip(inputCols).
    sorted.reverse.foreach(println)

(0.8176408960344523,Elevation)
(0.038934316703059905,Soil_Type_3)
(0.03078040510960056,Soil_Type_1)
(0.0277844260299692,Soil_Type_31)
(0.02558996898858634,Hillshade_Noon)
(0.01835509478117813,Horizontal_Distance_To_Hydriology)
(0.01472283138596372,Wilderness_Area_2)
(0.01072253211836195,Horizontal_Distance_To_Fire_Points)
(0.00791760083420736,Horizontal_Distance_To_Roadways)
(0.0034203438368014456,Soil_Type_2)
(0.003132419635918021,Wilderness_Area_0)
(9.991645419009118E-4,Soil_Type_9)
(0.0,Wilderness_Area_3)
(0.0,Wilderness_Area_1)
(0.0,Vertical_Distance_To_Hydrology)
(0.0,Soil_Type_8)
(0.0,Soil_Type_7)
(0.0,Soil_Type_6)
(0.0,Soil_Type_5)
(0.0,Soil_Type_4)
(0.0,Soil_Type_39)
(0.0,Soil_Type_38)
(0.0,Soil_Type_37)
(0.0,Soil_Type_36)
(0.0,Soil_Type_35)
(0.0,Soil_Type_34)
(0.0,Soil_Type_33)
(0.0,Soil_Type_32)
(0.0,Soil_Type_30)
(0.0,Soil_Type_29)
(0.0,Soil_Type_28)
(0.0,Soil_Type_27)
(0.0,Soil_Type_26)
(0.0,Soil_Type_25)
(0.0,Soil_Type_24)
(0.0,Soil_Type_23)
(0.0,Soil_Type_22)
(0.0,Soil_Ty

In [12]:
val predictions = model.transform(assembledTrainData)

predictions.select("Cover_Type","prediction","probability").
    show(truncate = false)

+----------+----------+-----------------------------------------------------------------------------------------------------------------+
|Cover_Type|prediction|probability                                                                                                      |
+----------+----------+-----------------------------------------------------------------------------------------------------------------+
|3.0       |3.0       |[0.0,3.391670058336725E-5,0.049789716456383123,0.6200312033645367,0.02204585537918871,0.0,0.3080993080993081,0.0]|
|3.0       |3.0       |[0.0,3.391670058336725E-5,0.049789716456383123,0.6200312033645367,0.02204585537918871,0.0,0.3080993080993081,0.0]|
|3.0       |3.0       |[0.0,3.391670058336725E-5,0.049789716456383123,0.6200312033645367,0.02204585537918871,0.0,0.3080993080993081,0.0]|
|6.0       |3.0       |[0.0,3.391670058336725E-5,0.049789716456383123,0.6200312033645367,0.02204585537918871,0.0,0.3080993080993081,0.0]|
|3.0       |3.0       |[0.0,3.3916

predictions: org.apache.spark.sql.DataFrame = [Elevation: int, Aspect: int ... 57 more fields]


In [13]:
val evaluator = new MulticlassClassificationEvaluator().
    setLabelCol("Cover_Type").
    setPredictionCol("prediction")

val accuracy = evaluator.setMetricName("accuracy").evaluate(predictions)

evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_5ba7811d2ce8
accuracy: Double = 0.7050595164088547


In [14]:
val predictionRDD = predictions.
    select("prediction","Cover_Type").
    as[(Double,Double)].rdd
val multiclassMetrics = new MulticlassMetrics(predictionRDD)
println(multiclassMetrics.confusionMatrix)

121508.0  66835.0   74.0     0.0     40.0   3.0    2253.0  
41945.0   209414.0  2812.0   33.0    334.0  41.0   250.0   
0.0       4606.0    27039.0  354.0   42.0   96.0   0.0     
0.0       8.0       1324.0   1141.0  0.0    0.0    0.0     
0.0       7926.0    276.0    0.0     411.0  0.0    0.0     
0.0       4973.0    10085.0  127.0   14.0   427.0  0.0     
9514.0    229.0     0.0      0.0     0.0    0.0    8663.0  


predictionRDD: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[89] at rdd at <console>:40
multiclassMetrics: org.apache.spark.mllib.evaluation.MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@5a64687a


In [15]:
val classifierHyper = new DecisionTreeClassifier().
setSeed(Random.nextLong()).
setLabelCol("Cover_Type").
setFeaturesCol("featureVector").
setPredictionCol("prediction")
val pipeline = new Pipeline().setStages(Array(assembler, classifierHyper))

classifierHyper: org.apache.spark.ml.classification.DecisionTreeClassifier = dtc_74d8eb61da53
pipeline: org.apache.spark.ml.Pipeline = pipeline_d3cfc3c2f311


In [16]:
val paramGrid = new ParamGridBuilder().
addGrid(classifierHyper.impurity, Seq("gini","entropy")).
addGrid(classifierHyper.maxDepth, Seq(1,5)).
addGrid(classifierHyper.maxBins, Seq(40,80)).
addGrid(classifierHyper.minInfoGain, Seq(0.0,0.05)).
build()

paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	dtc_74d8eb61da53-impurity: gini,
	dtc_74d8eb61da53-maxBins: 40,
	dtc_74d8eb61da53-maxDepth: 1,
	dtc_74d8eb61da53-minInfoGain: 0.0
}, {
	dtc_74d8eb61da53-impurity: gini,
	dtc_74d8eb61da53-maxBins: 80,
	dtc_74d8eb61da53-maxDepth: 1,
	dtc_74d8eb61da53-minInfoGain: 0.0
}, {
	dtc_74d8eb61da53-impurity: gini,
	dtc_74d8eb61da53-maxBins: 40,
	dtc_74d8eb61da53-maxDepth: 1,
	dtc_74d8eb61da53-minInfoGain: 0.05
}, {
	dtc_74d8eb61da53-impurity: gini,
	dtc_74d8eb61da53-maxBins: 80,
	dtc_74d8eb61da53-maxDepth: 1,
	dtc_74d8eb61da53-minInfoGain: 0.05
}, {
	dtc_74d8eb61da53-impurity: entropy,
	dtc_74d8eb61da53-maxBins: 40,
	dtc_74d8eb61da53-maxDepth: 1,
	dtc_74d8eb61da53-minInfoGain: 0.0
}, {
	dtc_74d8eb61da53-impurity: entropy,
	dtc_74d8eb6...

In [17]:
val multiclassEval = new MulticlassClassificationEvaluator().
setLabelCol("Cover_Type").
setPredictionCol("prediction").
setMetricName("accuracy")

val validator = new TrainValidationSplit().
setSeed(Random.nextLong()).
setEstimator(pipeline).
setEvaluator(multiclassEval).
setEstimatorParamMaps(paramGrid).
setTrainRatio(0.9)

val validatorModel = validator.fit(trainData)

multiclassEval: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_b7d94e3a779e
validator: org.apache.spark.ml.tuning.TrainValidationSplit = tvs_a301630f2fdd
validatorModel: org.apache.spark.ml.tuning.TrainValidationSplitModel = tvs_a301630f2fdd


In [18]:
val bestModel = validatorModel.bestModel
bestModel.asInstanceOf[PipelineModel].stages.last.extractParamMap

bestModel: org.apache.spark.ml.Model[_] = pipeline_d3cfc3c2f311
res9: org.apache.spark.ml.param.ParamMap =
{
	dtc_74d8eb61da53-cacheNodeIds: false,
	dtc_74d8eb61da53-checkpointInterval: 10,
	dtc_74d8eb61da53-featuresCol: featureVector,
	dtc_74d8eb61da53-impurity: gini,
	dtc_74d8eb61da53-labelCol: Cover_Type,
	dtc_74d8eb61da53-maxBins: 40,
	dtc_74d8eb61da53-maxDepth: 5,
	dtc_74d8eb61da53-maxMemoryInMB: 256,
	dtc_74d8eb61da53-minInfoGain: 0.0,
	dtc_74d8eb61da53-minInstancesPerNode: 1,
	dtc_74d8eb61da53-predictionCol: prediction,
	dtc_74d8eb61da53-probabilityCol: probability,
	dtc_74d8eb61da53-rawPredictionCol: rawPrediction,
	dtc_74d8eb61da53-seed: 2196387991364154618
}


In [19]:
val paramAndMetrics = validatorModel.validationMetrics.
zip(validatorModel.getEstimatorParamMaps).sortBy(-_._1)
paramAndMetrics.foreach{ case (metric, params) => 
println(metric)
println(params)
println()
}

0.7037961079338272
{
	dtc_74d8eb61da53-impurity: gini,
	dtc_74d8eb61da53-maxBins: 40,
	dtc_74d8eb61da53-maxDepth: 5,
	dtc_74d8eb61da53-minInfoGain: 0.0
}

0.7028365255440833
{
	dtc_74d8eb61da53-impurity: gini,
	dtc_74d8eb61da53-maxBins: 80,
	dtc_74d8eb61da53-maxDepth: 5,
	dtc_74d8eb61da53-minInfoGain: 0.0
}

0.702721375657314
{
	dtc_74d8eb61da53-impurity: entropy,
	dtc_74d8eb61da53-maxBins: 80,
	dtc_74d8eb61da53-maxDepth: 5,
	dtc_74d8eb61da53-minInfoGain: 0.0
}

0.6987678962115688
{
	dtc_74d8eb61da53-impurity: entropy,
	dtc_74d8eb61da53-maxBins: 40,
	dtc_74d8eb61da53-maxDepth: 5,
	dtc_74d8eb61da53-minInfoGain: 0.0
}

0.6918972863010018
{
	dtc_74d8eb61da53-impurity: entropy,
	dtc_74d8eb61da53-maxBins: 80,
	dtc_74d8eb61da53-maxDepth: 5,
	dtc_74d8eb61da53-minInfoGain: 0.05
}

0.6883084481633593
{
	dtc_74d8eb61da53-impurity: entropy,
	dtc_74d8eb61da53-maxBins: 40,
	dtc_74d8eb61da53-maxDepth: 5,
	dtc_74d8eb61da53-minInfoGain: 0.05
}

0.6706713238398649
{
	dtc_74d8eb61da53-impurity: gini,
	d

paramAndMetrics: Array[(Double, org.apache.spark.ml.param.ParamMap)] =
Array((0.7037961079338272,{
	dtc_74d8eb61da53-impurity: gini,
	dtc_74d8eb61da53-maxBins: 40,
	dtc_74d8eb61da53-maxDepth: 5,
	dtc_74d8eb61da53-minInfoGain: 0.0
}), (0.7028365255440833,{
	dtc_74d8eb61da53-impurity: gini,
	dtc_74d8eb61da53-maxBins: 80,
	dtc_74d8eb61da53-maxDepth: 5,
	dtc_74d8eb61da53-minInfoGain: 0.0
}), (0.702721375657314,{
	dtc_74d8eb61da53-impurity: entropy,
	dtc_74d8eb61da53-maxBins: 80,
	dtc_74d8eb61da53-maxDepth: 5,
	dtc_74d8eb61da53-minInfoGain: 0.0
}), (0.6987678962115688,{
	dtc_74d8eb61da53-impurity: entropy,
	dtc_74d8eb61da53-maxBins: 40,
	dtc_74d8eb61da53-maxDepth: 5,
	dtc_74d8eb61da53-minInfoGain: 0.0
}), (0.6918972863010018,{
	dtc_74d8eb61da53-impurity: entropy,
	dtc_74d8eb61da53-maxBins: 8...

In [20]:
validatorModel.validationMetrics.max
multiclassEval.evaluate(bestModel.transform(testData))

res11: Double = 0.7053680322940823
